In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip3 install torchvision

In [0]:
!pip3 install Pillow

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch
from torch.autograd import Variable
import utilss as utils
import dataset 
from PIL import Image

import crnn as crnn

#'drive/My Drive/Colab Notebooks/TRABAJO/pytorch/'
model_path = 'drive/My Drive/Colab Notebooks/TRABAJO/pytorch/data/crnn.pth'
img_path = 'drive/My Drive/Colab Notebooks/TRABAJO/pytorch/data/demo5.PNG'
alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'

model = crnn.CRNN(32, 1, 37, 256)
if torch.cuda.is_available():
    model = model.cuda()
print('loading pretrained model from %s' % model_path)
model.load_state_dict(torch.load(model_path))

converter = utils.strLabelConverter(alphabet)

transformer = dataset.resizeNormalize((100, 32))
image = Image.open(img_path).convert('L')
image = transformer(image)
if torch.cuda.is_available():
    image = image.cuda()
image = image.view(1, *image.size())
image = Variable(image)

model.eval()
preds = model(image)

_, preds = preds.max(2)
preds = preds.transpose(1, 0).contiguous().view(-1)

preds_size = Variable(torch.IntTensor([preds.size(0)]))
raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
print('%-20s => %-20s' % (raw_pred, sim_pred))


loading pretrained model from drive/My Drive/Colab Notebooks/TRABAJO/pytorch/data/crnn.pth
c-----z-------5-----s----- => cz5s                
